In [ ]:

# @title SETUP
!pip install git+https://github.com/huggingface/diffusers.git transformers xformers accelerate omegaconf torchsde
!apt -y install -qq aria2

In [ ]:
import os
os.makedirs(f'models',exist_ok=True)

# @title DOWNLOAD MODEL

Model_url = 'https://civitai.com/api/download/models/155282?type=Model&format=SafeTensor&size=pruned&fp=fp16'# @param {type:"string"}
Model_name = 'Sassy_Girls'# @param {type:"string"}
if not Model_name.endswith('.safetensors'):
    Model_name=Model_name+'.safetensors'

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Model_url}" -d models -o {Model_name}
print('\nAvailable models :')
for MODEL in os.listdir('models'):
    print(MODEL.replace('.safetensors',''))

EMBEDDING_URL SHOULD BE A URL TO A '.safetensors' FILE

In [ ]:
# @title LOAD EMBEDDING

EMBEDDING_URL= ''# @param {type:"string"}
embed = req.get(EMBEDDING_URL)
filename = embed.headers['Content-Disposition'].split('filename=')[1].strip('"')
tk = filename.strip('.safetensors')
with open(filename,"wb") as r:
    r.write(embed.content)
state_dict = load_file(filename)
try:
    pipe.load_textual_inversion(state_dict["clip_l"], token=tk, text_encoder=pipe.text_encoder, tokenizer=pipe.tokenizer)
    print(f"Successfuly Loaded\n\nTrigger Keyword :\n{tk}")
except Exception as e:
    em = str(e)
    if f"Token {tk} already" in em:
        print(f"Successfuly Loaded\n\nTrigger Keyword :\n{tk}")
    else:
        print(em)

LOADING MANY LORAs IS NOT SUGGESTED AS COLAB HAS LIMITED RESOURCES AND IT MIGHT CRASH , 1-2 LORAs WORK FINE

In [ ]:
# @title LOAD LORA
LORA_NAME= ''# @param {type:"string"}
LORA_SCALE  = 0.5# @param {type:"number"}
if not LORA_NAME.endswith('.safetensors'):
    LORA_NAME=LORA_NAME+'.safetensors'
print(f'\n\nCurrently selected : {LORA_NAME}')
lora_path = f'models/{LORA_NAME}'
pipe.load_lora_weights(lora_path)
pipe.fuse_lora(1)

In [ ]:
# @title LOAD MODEL

from diffusers import StableDiffusionPipeline, DDIMParallelScheduler, DDIMScheduler, DDPMParallelScheduler, DDPMScheduler, DEISMultistepScheduler, DPMSolverMultistepScheduler, DPMSolverSDEScheduler, DPMSolverSinglestepScheduler, EulerAncestralDiscreteScheduler, EulerDiscreteScheduler, HeunDiscreteScheduler, KDPM2AncestralDiscreteScheduler, KDPM2DiscreteScheduler, LMSDiscreteScheduler, PNDMScheduler, UniPCMultistepScheduler
from diffusers.utils import make_image_grid
import torch,random,os
from IPython.display import display

os.makedirs(f'outputs',exist_ok=True)
MODEL_NAME  = "Sassy_Girls"# @param {type:"string"}
LOAD_TYPE = "FP16 (FAST BUT AVERAGE QUALITY)"# @param ["FP32 (SLOW BUT GOOD QUALITY)","FP16 (FAST BUT AVERAGE QUALITY)"]
if not MODEL_NAME.endswith('.safetensors'):
    MODEL_NAME=MODEL_NAME+'.safetensors'
print(f'\n\nCurrently selected : {MODEL_NAME}')
model_path = f'models/{MODEL_NAME}'
if LOAD_TYPE == "FP32 (SLOW BUT GOOD QUALITY)":
    pipe = StableDiffusionPipeline.from_single_file(model_path,load_safety_checker = False,scheduler_type='dpm')
else:
    pipe = StableDiffusionPipeline.from_single_file(model_path, revision="fp16",load_safety_checker = False, torch_dtype=torch.float16,variant="fp16",scheduler_type='dpm')
pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# @title GENERATE IMAGE
from PIL import PngImagePlugin

PROMPT= ''# @param {type:"string"}
NEGATIVE_PROMPT = ''# @param {type:"string"}
WIDTH  = 512# @param {type:"integer"}
HEIGHT = 512 # @param {type:"integer"}
SAMPLING_STEPS  = 20# @param {type:"integer"}
CFG_scale  = 7# @param {type:"number"}
SEED  = -1# @param {type:"number"}
CLIP_SKIP  = 1# @param {type:"number"}
SCHEDULER  = "DPMSolverMultistepScheduler"# @param ["DDIMParallelScheduler","DDIMScheduler","DDPMParallelScheduler","DDPMScheduler","DEISMultistepScheduler","DPMSolverMultistepScheduler","DPMSolverSDEScheduler","DPMSolverSinglestepScheduler","EulerAncestralDiscreteScheduler","EulerDiscreteScheduler","HeunDiscreteScheduler","KDPM2AncestralDiscreteScheduler","KDPM2DiscreteScheduler","LMSDiscreteScheduler","PNDMScheduler","UniPCMultistepScheduler"]
USE_KARRAS = True # @param {type:"boolean"}
NUMBER_OF_IMAGES  = 1# @param {type:"number"}
CLIP_SKIP = None if CLIP_SKIP == 1 else CLIP_SKIP - 1

# Schedulers mapping: https://huggingface.co/docs/diffusers/api/schedulers/overview
def get_a1111_name(scheduler, use_karras):
    diffusers_to_a1111_map = {
        "DPMSolverMultistepScheduler": {
            False: "DPM++ 2M",
            True: "DPM++ 2M Karras"
        },
        "DPMSolverSinglestepScheduler": {
            False: "DPM++ SDE",
            True: "DPM++ SDE Karras"
        },
        "KDPM2DiscreteScheduler": {
            False: "DPM2",
            True: "DPM2 Karras"
        },
        "KDPM2AncestralDiscreteScheduler": {
            False: "DPM2 a",
            True: "DPM2 a Karras"
        },
        "EulerDiscreteScheduler": {
            False: "Euler"
        },
        "EulerAncestralDiscreteScheduler": {
            False: "Euler a"
        },
        "HeunDiscreteScheduler": {
            False: "Heun"
        },
        "LMSDiscreteScheduler": {
            False: "LMS",
            True: "LMS Karras"
        },
        "DEISMultistepScheduler": {
            False: "N/A"
        },
        "UniPCMultistepScheduler": {
            False: "N/A"
        }
    }
    
    return diffusers_to_a1111_map.get(scheduler, {}).get(use_karras, scheduler)


def embed_png_info(image_path, info):
    with PngImagePlugin.PngImageFile(image_path) as img:
        # Embed the info as a PNG text chunk
        meta = PngImagePlugin.PngInfo()
        #encoded_info = b'UNICODE' + info.encode('utf-8')
        meta.add_text("parameters", info)
        img.save(image_path, pnginfo=meta)

def format_png_info(prompt, negative_prompt, steps, width, height, seed, scheduler, cfg_scale, clip_skip,**kwargs):
    # Construct the PNG info
    info = prompt + "\n"
    info += f"Negative prompt: {negative_prompt}\n"
    info += f"Steps: {steps}, Size: {width}x{height}, Seed: {seed}, Sampler: {scheduler}, CFG scale: {cfg_scale}, Clip skip: {clip_skip},\n"

    # Add any additional kwargs
    for key, value in kwargs.items():
        info += f"{key}: {value}, "

    # Remove trailing comma and space
    info = info.rstrip(", ")
    
    return info

def save_image_with_png_info(image_path):
    global PROMPT
    global NEGATIVE_PROMPT
    global SAMPLING_STEPS
    global WIDTH
    global HEIGHT
    global SEED
    global SCHEDULER
    global USE_KARRAS
    global CFG_scale
    global CLIP_SKIP
    
    clip_skip = CLIP_SKIP
    if clip_skip is None:
        clip_skip = 1
    
    # Save the image
    image.save(image_path)

    # Generate PNG info and embed it into the image
    embed_png_info(image_path, 
                   format_png_info(PROMPT, NEGATIVE_PROMPT, SAMPLING_STEPS,
                                   WIDTH, HEIGHT, SEED, get_a1111_name(SCHEDULER, USE_KARRAS), CFG_scale, 
                                   clip_skip ) )

                   
if SEED == -1:
    SEED = random.randint(1,10000000000)
generator = torch.Generator(device="cuda").manual_seed(SEED)
sc = f'''pipe.scheduler = {SCHEDULER}.from_config(pipe.scheduler.config, use_karras_sigmas={USE_KARRAS})'''
exec(sc)
final_images=[]
if NUMBER_OF_IMAGES != 1:
    for i in range(NUMBER_OF_IMAGES):
        SEED = random.randint(1,10000000000)
        print(f'SEED : {SEED}')
        generator = torch.Generator(device="cuda").manual_seed(SEED)
        image = pipe(PROMPT,negative_prompt = NEGATIVE_PROMPT,clip_skip=CLIP_SKIP,generator=generator,width = WIDTH , height = HEIGHT,num_inference_steps=SAMPLING_STEPS,guidance_scale = CFG_scale).images[0]
        final_images.append(image)
        save_image_with_png_info(f'outputs/{PROMPT[:10]}_{SEED}.png')
    f_images = make_image_grid(final_images, 1, len(final_images))
    display(f_images)
else:
    print(f'SEED : {SEED}')
    image = pipe(PROMPT,negative_prompt = NEGATIVE_PROMPT,clip_skip=CLIP_SKIP,generator=generator,width = WIDTH , height = HEIGHT,num_inference_steps=SAMPLING_STEPS,guidance_scale = CFG_scale).images[0]
    save_image_with_png_info(f'outputs/{PROMPT[:11]}_{SEED}_{CLIP_SKIP}_{SCHEDULER}.png')
    display(image)


In [ ]:
# @title DOWNLOAD THE ZIP WITH ALL THE IMAGES
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

folder_to_zip = "outputs"
zip_filename = "output.zip"
zip_folder(folder_to_zip, zip_filename)
print(f'Succesfully saved all the images in {zip_filename}\nDownloading the zip.....')
files.download(zip_filename)